## Import Modul

In [15]:
import librosa
import numpy as np
import os
from tensorflow.image import resize
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

In [16]:
# Define folder paths and classes

data_dir = os.path.join(os.path.dirname(os.getcwd()), "Dataset")
classes = ["Aedes Aegypti", "Anopheles Stephensi", "Culex Pipiens"]

In [17]:
def augment_audio(audio_data, sample_rate):
    """
    Augments audio data with noise, pitch shifting, time stretching, etc.

    Parameters:
    audio_data (numpy array): The raw audio data.
    sample_rate (int): The sample rate of the audio data.

    Returns:
    list: A list of augmented audio data.
    """
    # Adding noise
    noise = np.random.randn(len(audio_data))
    audio_data_noisy = audio_data + 0.005 * noise

    # Pitch shifting
    audio_data_pitch = librosa.effects.pitch_shift(audio_data, sr=sample_rate, n_steps=4)

    # Time stretching
    audio_data_stretch = librosa.effects.time_stretch(audio_data, rate=1.2)

    # Random shifting (time translation)
    max_shift = int(0.1 * len(audio_data))  # 10% of audio length
    shift = np.random.randint(-max_shift, max_shift)
    audio_data_shifted = np.roll(audio_data, shift)

    # Random volume change
    gain_factor = np.random.uniform(0.7, 1.3)  # 70% to 130% gain
    audio_data_gain = audio_data * gain_factor

    # Reverberation (simplified by pre-emphasis)
    audio_data_reverb = librosa.effects.preemphasis(audio_data)

    return [audio_data_noisy, audio_data_pitch, audio_data_stretch, audio_data_shifted, audio_data_gain, audio_data_reverb]

def chunk_audio(audio_data, sample_rate, chunk_duration=3, overlap_duration=1):
    """
    Splits audio data into chunks.

    Parameters:
    audio_data (numpy array): The audio data to be chunked.
    sample_rate (int): The sample rate of the audio data.
    chunk_duration (int): Duration of each chunk in seconds.
    overlap_duration (int): Duration of overlap between chunks in seconds.

    Returns:
    list: A list of audio chunks.
    """
    chunk_samples = chunk_duration * sample_rate
    overlap_samples = overlap_duration * sample_rate

    num_chunks = int((len(audio_data) - chunk_samples) / (chunk_samples - overlap_samples)) + 1
    chunks = []

    for i in range(num_chunks):
        start = i * (chunk_samples - overlap_samples)
        end = start + chunk_samples
        chunks.append(audio_data[start:end])

    return chunks

In [18]:
def load_and_preprocess_data(data_dir, classes, target_shape=(180, 180)):
    """
    Loads and preprocesses audio data, including augmentation and feature extraction.

    Parameters:
    data_dir (str): The directory containing the dataset.
    classes (list): A list of class labels.
    target_shape (tuple): The target shape for resizing the spectrograms.

    Returns:
    numpy array, numpy array: Preprocessed data and corresponding labels.
    """
    data = []
    labels = []

    for i_class, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        print(f"Processing {class_name}...")
        for filename in os.listdir(class_dir):
            if filename.endswith(".wav"):
                file_path = os.path.join(class_dir, filename)
                audio_data, sample_rate = librosa.load(file_path, sr=None)

                # Augment audio data
                augmented_data = augment_audio(audio_data, sample_rate)

                # Chunk the augmented audio data
                for audio in augmented_data:
                    chunks = chunk_audio(audio, sample_rate)
                    
                    for chunk in chunks:
                        # Extract features: Mel Spectrogram and MFCC
                        mel_spectrogram = librosa.feature.melspectrogram(y=chunk, sr=sample_rate, n_mels=128)
                        mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
                        mel_spectrogram_resized = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)

                        mfcc = librosa.feature.mfcc(y=chunk, sr=sample_rate, n_mfcc=13)
                        mfcc_resized = resize(np.expand_dims(mfcc, axis=-1), target_shape)

                        # Combine features and append to data and labels
                        combined_features = np.concatenate([mel_spectrogram_resized, mfcc_resized], axis=-1)
                        data.append(combined_features)
                        labels.append(i_class)

    return np.array(data), np.array(labels)

In [ ]:
# Load and preprocess the data

data, labels = load_and_preprocess_data(data_dir, classes)

Processing Aedes Aegypti...
Processing Anopheles Stephensi...
Processing Culex Pipiens...


In [20]:
data.shape, labels.shape

((1905, 180, 180, 2), (1905,))

In [21]:
# One-hot encode labels
labels = to_categorical(labels, num_classes=len(classes))

labels

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [22]:
# Compute class weights
y_labels = np.argmax(labels, axis=1)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_labels), y=y_labels)
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

# Menampilkan class weight
print("Class Weights: ", class_weight_dict)

Class Weights:  {0: 1.0478547854785478, 1: 0.9071428571428571, 2: 1.0601001669449082}


In [23]:
# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [25]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1524, 180, 180, 2), (381, 180, 180, 2), (1524, 3), (381, 3))

In [26]:
X_train

array([[[[-7.9996948e+01, -5.7313220e+02],
         [-7.9995247e+01, -5.5195453e+02],
         [-8.0000000e+01, -5.6657074e+02],
         ...,
         [-8.0000000e+01, -6.2565338e+02],
         [-8.0000000e+01, -6.2565796e+02],
         [-8.0000000e+01, -6.2565796e+02]],

        [[-7.9998680e+01, -5.7313220e+02],
         [-7.9997940e+01, -5.5195453e+02],
         [-8.0000000e+01, -5.6657074e+02],
         ...,
         [-8.0000000e+01, -6.2565338e+02],
         [-8.0000000e+01, -6.2565796e+02],
         [-8.0000000e+01, -6.2565796e+02]],

        [[-8.0000000e+01, -5.7313220e+02],
         [-8.0000000e+01, -5.5195453e+02],
         [-8.0000000e+01, -5.6657074e+02],
         ...,
         [-8.0000000e+01, -6.2565338e+02],
         [-8.0000000e+01, -6.2565796e+02],
         [-8.0000000e+01, -6.2565796e+02]],

        ...,

        [[-8.0000000e+01, -1.5032070e+01],
         [-8.0000000e+01, -1.0937807e+01],
         [-8.0000000e+01,  1.2063111e+00],
         ...,
         [-8.0000000e

In [27]:
X_test

array([[[[-5.44247398e+01, -5.47978455e+02],
         [-5.44247398e+01, -5.47978455e+02],
         [-5.47812576e+01, -5.39328735e+02],
         ...,
         [-5.86506996e+01, -5.69973999e+02],
         [-5.83859215e+01, -5.76744568e+02],
         [-5.83859215e+01, -5.76744568e+02]],

        [[-5.49562988e+01, -5.47978455e+02],
         [-5.49562988e+01, -5.47978455e+02],
         [-5.56008072e+01, -5.39328735e+02],
         ...,
         [-6.00781021e+01, -5.69973999e+02],
         [-5.94589233e+01, -5.76744568e+02],
         [-5.94589233e+01, -5.76744568e+02]],

        [[-5.54380417e+01, -5.47978455e+02],
         [-5.54380417e+01, -5.47978455e+02],
         [-5.63123283e+01, -5.39328735e+02],
         ...,
         [-5.92190628e+01, -5.69973999e+02],
         [-5.84281349e+01, -5.76744568e+02],
         [-5.84281349e+01, -5.76744568e+02]],

        ...,

        [[-6.30783081e+01,  1.00568533e-02],
         [-6.30783081e+01,  1.00568533e-02],
         [-6.39057350e+01,  2.09153414

In [28]:
y_train

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [29]:
y_test

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [30]:
# Save preprocessed data and labels

# Tentukan path absolut ke folder tujuan
output_dir = os.path.join(os.path.dirname(os.getcwd()), "Output")

# Pastikan folder tujuan ada
if not os.path.exists(output_dir):
    os.makedirs(output_dir)  # Membuat folder jika belum ada

# Simpan file di dalam folder tersebut
np.save(os.path.join(output_dir, "X_train.npy"), X_train)
np.save(os.path.join(output_dir, "X_test.npy"), X_test)
np.save(os.path.join(output_dir, "y_train.npy"), y_train)
np.save(os.path.join(output_dir, "y_test.npy"), y_test)
np.save(os.path.join(output_dir, "class_weights.npy"), class_weight_dict)